In [84]:
import pandas as pd
from pathlib import Path
import numpy as np
import itertools

def get_row_from_metadata(metadata, covariate_name):
    """
    Extracts a specific row from the metadata DataFrame based on the covariate name.

    :param metadata: DataFrame containing metadata.
    :param covariate_name: Name of the covariate to extract.
    :return: Row corresponding to the specified covariate name.
    """
    return metadata.loc[metadata['variable_name'] == covariate_name].squeeze()

def all_rows_from_metadata_containing(metadata, substring):
    """
    Extracts all rows from the metadata DataFrame that contain a specific substring in the variable name.

    :param metadata: DataFrame containing metadata.
    :param substring: Substring to search for in the variable names.
    :return: DataFrame containing all rows with variable names that contain the substring.
    """
    return metadata[metadata['variable_name'].str.contains(substring, na=False)].reset_index(drop=True)

def all_column_names_containing(df, substring):
    """
    Extracts all column names from the DataFrame that contain a specific substring.

    :param df: DataFrame to search for column names.
    :param substring: Substring to search for in the column names.
    :return: List of column names containing the specified substring.
    """
    return [col for col in df.columns if substring in col]


def find_equivalent_columns(data, summary, numeric_tolerance=1e-6, categorical_threshold=0.99):
    """
    Find pairs of columns in a DataFrame that are informationally equivalent.
    
    Parameters:
    -----------
    data : pandas DataFrame
        The DataFrame to analyze
    numeric_tolerance : float, default 1e-6
        Tolerance for considering numeric columns equal or proportional
    categorical_threshold : float, default 0.99
        Threshold for considering categorical columns equivalent (percentage match)
    
    Returns:
    --------
    list of tuples
        Each tuple contains (col1, col2, relationship_type)
        where relationship_type is one of: 'identical', 'proportional', 'categorical_equivalent'
    """
    equivalent_pairs = []
    columns = data.columns
    
    # Get column types
    
    numeric_cols = summary[summary.data_type == 'numeric'].variable_name.tolist()
    categorical_cols = summary[summary.data_type == 'categorical'].variable_name.tolist()

    # Identify constant columns
    constant_cols = []
    for col in columns:
        unique_values = data[col].dropna().unique()
        if len(unique_values) <= 1:
            constant_cols.append(col)

    # Print constant columns if verbose
    if len(constant_cols) > 0:
        print("Constant columns:")
        for col in constant_cols:
            print(col)
        print()
    

    # Remove constant columns from numeric and categorical lists
    numeric_cols = [col for col in numeric_cols if col not in constant_cols]
    categorical_cols = [col for col in categorical_cols if col not in constant_cols]

    # remove missingness-indicator columns
    missingness_cols = list(
        set(all_column_names_containing(data, '_missing') + 
        all_column_names_containing(data, '_m'))
    )
    numeric_cols = [col for col in numeric_cols if col not in missingness_cols]
    categorical_cols = [col for col in categorical_cols if col not in missingness_cols]
    
    # Check numeric columns for equality or proportionality
    for col1, col2 in itertools.combinations(numeric_cols, 2):

        # Check for identical values first
        if data[col1].equals(data[col2]):
            equivalent_pairs.append((col1, col2, 'identical'))
            continue
            
        # Check for identical values where neither is zero
        valid_mask = ~data[col1].isna() & ~data[col2].isna()
        if np.allclose(data.loc[valid_mask, col1], data.loc[valid_mask, col2], 
                      rtol=numeric_tolerance, atol=numeric_tolerance):
            equivalent_pairs.append((col1, col2, 'nearly_identical'))
            continue
        
        # For rows with zeros, check if the columns are exactly equal
        zero_mask = (data[col1] == 0) | (data[col2] == 0)
        non_zero_mask = ~zero_mask & valid_mask
        
        # Check if the columns have the same values where zeros are present
        if zero_mask.any():
            zero_equality = (data.loc[zero_mask & valid_mask, col1] == 
                             data.loc[zero_mask & valid_mask, col2]).all()
        else:
            zero_equality = True
            
        # Check for proportional relationship in non-zero values
        if non_zero_mask.sum() > 10:  # Require at least some non-zero values
            ratios = data.loc[non_zero_mask, col2] / data.loc[non_zero_mask, col1]
            ratio_std = ratios.std()
            
            # If standard deviation of ratios is very small, columns are proportional
            if ratio_std < numeric_tolerance and zero_equality:
                ratio = ratios.mean()
                equivalent_pairs.append((col1, col2, f'proportional (factor: {ratio:.4f})'))
    
    # Create a list of all columns to check for categorical equivalence
    # This includes both explicit categorical columns and numeric columns
    all_potential_categorical_cols = categorical_cols + numeric_cols
    
    # Check all columns for equivalent categorical mappings
    for col1, col2 in itertools.combinations(all_potential_categorical_cols, 2):
        # Skip if identical columns or already identified as identical or proportional
        if col1 == col2 or any((col1, col2, rel) in equivalent_pairs for rel in 
                               ['identical', 'nearly_identical', 'proportional']):
            continue
            
        # Get unique values for both columns
        unique_vals1 = data[col1].dropna().unique()
        unique_vals2 = data[col2].dropna().unique()
        
        # Skip if columns have different number of unique values
        if len(unique_vals1) != len(unique_vals2):
            continue
            
        # Skip if too many unique values (likely not categorical)
        if len(unique_vals1) > 100:  # Arbitrary threshold, adjust as needed
            continue
            
        # Create a mapping table between values in both columns
        mapping_df = data[[col1, col2]].dropna().drop_duplicates()
        
        # Check if mapping is one-to-one (each value in col1 maps to exactly one value in col2)
        is_one_to_one = True
        
        # Check col1 -> col2 mapping
        for val in unique_vals1:
            corresponding_vals = data.loc[data[col1] == val, col2].dropna().unique()
            if len(corresponding_vals) != 1:
                is_one_to_one = False
                break
                
        # Check col2 -> col1 mapping
        if is_one_to_one:
            for val in unique_vals2:
                corresponding_vals = data.loc[data[col2] == val, col1].dropna().unique()
                if len(corresponding_vals) != 1:
                    is_one_to_one = False
                    break
        
        if is_one_to_one:
            # If we create a new column using the mapping, it should match the original
            val_mapping = dict(zip(mapping_df[col1], mapping_df[col2]))
            
            # Apply mapping and handle NaN values
            mapped_values = data[col1].map(val_mapping)
            
            # Count matches (ignoring NaN values)
            valid_mask = ~data[col1].isna() & ~data[col2].isna()
            if valid_mask.sum() > 0:
                match_percentage = (mapped_values == data[col2])[valid_mask].mean()
                
                if match_percentage >= categorical_threshold:
                    # Determine if both are numeric or mixed types
                    if col1 in numeric_cols and col2 in numeric_cols:
                        relationship = 'numeric_categorical_equivalent'
                    else:
                        relationship = 'categorical_equivalent'
                    equivalent_pairs.append((col1, col2, relationship))
    
    return equivalent_pairs

"""
Done in this notebook
- Ensure that missingness-indicator columns exist.
    - You probably can't conclusively check that all are included, because the data you get will not necessarily reveal which columns had missingness, but check that there are some missingness columns, and none for categorical data.
- Ensure there are no NaNs in the data.
- Ensure column names:
    - In data: "hhid" (if household ID is included), "consumption_per_capita_per_day", "hh_wgt".
    - Consumption: Check mean and std for sanity. In a poor country, the mean should be low-mid single digits: e.g., in Uganda, the mean is $3.80/day.
- Check for columns that indicate units:
    - If they are present, the corresponding numeric field should be standardized, e.g., all area units adjusted to square meters.
- Check that metadata and the dataset itself match:
    - Every column in data is described in metadata and vice versa. It's also OK if `hhid` is not in the data at all.
- In metadata:
    - "variable_name".
    - "data_type", with permitted values "numeric" and "categorical".
    - "geographic_indicator".
- Scan datatypes:
    - In particular, make sure nothing is numeric which should be categorical.
    - Ensure categorical-type columns have the appropriate type even if the categories are encoded as integers (if a column is binary, with no missing values, it can be numeric or categorical).
    - IDs of all kinds are strings even if they appear numeric.
- Check for duplication
- Check feasibility of stratification
"""

'\nDone in this notebook\n- Ensure that missingness-indicator columns exist.\n    - You probably can\'t conclusively check that all are included, because the data you get will not necessarily reveal which columns had missingness, but check that there are some missingness columns, and none for categorical data.\n- Ensure there are no NaNs in the data.\n- Ensure column names:\n    - In data: "hhid" (if household ID is included), "consumption_per_capita_per_day", "hh_wgt".\n    - Consumption: Check mean and std for sanity. In a poor country, the mean should be low-mid single digits: e.g., in Uganda, the mean is $3.80/day.\n- Check for columns that indicate units:\n    - If they are present, the corresponding numeric field should be standardized, e.g., all area units adjusted to square meters.\n- Check that metadata and the dataset itself match:\n    - Every column in data is described in metadata and vice versa. It\'s also OK if `hhid` is not in the data at all.\n- In metadata:\n    - "va

## Read in

In [85]:
# not used except to print - a sanity check in my script which I use for many countries.
country = 'Guatemala'

data = pd.read_csv('../merged_data/guatemala_household_data.csv')
summary = pd.read_csv('../metadata/guatemala_metadata_final.csv')

if 'variable_description' not in summary.columns:
    summary['variable_description'] = summary['variable_name']

print(f'Read in: {country}')

Read in: Guatemala


In [86]:
# It's critical there are no Nones or nans.

print(f'country: {country}')
print('nullity: ')
display(data.isna().mean().sort_values(ascending=False).head(2))
# Empty string may or may not be a problem; just take a glance at whether they seem to make sense.

print('empty string')
display(data.isin(['']).mean().sort_values(ascending=False).head(2))
print('Number of samples:')
print(data.shape[0])

country: Guatemala
nullity: 


region    0.0
area      0.0
dtype: float64

empty string


region    0.0
area      0.0
dtype: float64

Number of samples:
7276


## Missingness columns

In [87]:
# Missingness columns (assumes _missing or _m suffix)
# We've asked everyone to include missingness indicators for all numeric columns. If there are numeric columns without
# indicators, ensure that makes sense: e.g. counts, where missing data would result in a zero, is fine. 

# If there are categorical columns with missingess indicators, that's incorrect. If it is trivial for you to fix in their code,
# great, but if it'll take more than a minute or two, just note the issue and move on.

print(f'country: {country}')

missingness_columns_missing = [
    c for c in data.columns if ('missing' in c) 
]
missingness_columns_m = [
    c for c in data.columns if ('_m' in c) 
]
with_missingness = [
    c[:-8] for c in missingness_columns_missing
] + [
    c[:-2] for c in missingness_columns_m
]
missingness_columns = missingness_columns_missing + missingness_columns_m
for c in missingness_columns:
    if not (c in summary.variable_name.values):
        print(f"Missingness column {c} not in summary")
    
relevant_summary = summary[summary.variable_name.isin(with_missingness)]
print('categorical columns with missingness indicators:')

print(relevant_summary.data_type.value_counts())
display(relevant_summary[relevant_summary.data_type == 'categorical'])

# print numerical columns with no missingness indicators
print('numerical columns with no missingness indicators:')
print(summary[
    (summary.data_type == 'numeric') 
    & (~summary.variable_name.isin(with_missingness))
    & (~summary.variable_name.str.endswith('_missing'))
    & (~summary.variable_name.str.endswith('_m'))
].variable_name)

country: Guatemala
categorical columns with missingness indicators:
Series([], Name: count, dtype: int64)


,variable_name,data_type,module_name,module_description,variable_description,geographic_indicator


numerical columns with no missingness indicators:
2                           hhid
3                        hh_size
4                         hh_wgt
5                  num_gas_stove
6                  num_microwave
                 ...            
163                   num_adults
164                  num_elderly
166                  hh_head_age
173        num_children_enrolled
182    headcount_adjusted_hh_wgt
Name: variable_name, Length: 112, dtype: object


## Consumption, weights, hh size, poverty rate

In [88]:
# Here, just ensure the assertions pass, and then send me the poverty rate that is printed (not the mean or the 
# standard deviation).

print(f'country: {country}')
assert 'consumption_per_capita_per_day' in data.columns
assert 'headcount_adjusted_hh_wgt' in data.columns
assert pd.api.types.is_numeric_dtype(data['consumption_per_capita_per_day']), "'consumption_per_capita_per_day' is not numeric"
assert pd.api.types.is_numeric_dtype(data['headcount_adjusted_hh_wgt']), "'headcount_adjusted_hh_wgt' is not numeric"
if not 'hh_size' in data.columns:
    print('Warning: Missing hh_size')
else:
    assert np.isclose(data.hh_size * data.hh_wgt, data.headcount_adjusted_hh_wgt).all()
    assert pd.api.types.is_numeric_dtype(data['hh_size']), "'hh_size' is not numeric"

for col in ['headcount_adjusted_hh_wgt_missing', 'consumption_per_capita_per_day_missing', 'hh_wgt_missing']:
    if col in data.columns:
        assert data[col].sum() == 0, f"{col} has missing values"

print('mean:', data.consumption_per_capita_per_day.mean())
print('std:', data.consumption_per_capita_per_day.std())

count_poor = (
    data[data.consumption_per_capita_per_day < 2.15].headcount_adjusted_hh_wgt
).sum()

total = (
    data.headcount_adjusted_hh_wgt
).sum()
rate = count_poor / total

print('rate:',rate)
# To crosscheck: https://docs.google.com/spreadsheets/d/11wGVZadIZMvR2oXoDtSfjJVvixyv3ievuUOF4k_1HNY/edit?gid=0#gid=0

country: Guatemala
mean: 14.123801400744423
std: 16.30854291105592
rate: 0.025520301311164544


## Suspiciously named columns

In [89]:
# This section checks for common pitfalls.

print(f'country: {country}')

# Data which might be a unit for some other data (we don't want this)
#   e.g. "units of plot area"
print('containing the word "unit":')
display(
    summary[
        (
            summary.variable_name.str.contains('unit')
            | summary.variable_description.str.contains('unit')
        ) & (
            ~summary.variable_name.str.contains('community')
        )
    ]
)

# The only consumption data should be "consumption_per_capita_per_day"
print('containing the word "consumption":')
display(
    summary[
        summary.variable_name.str.contains('consumption')
        | summary.variable_description.str.contains('consumption')
    ]
)

# Print variables whose name contains "id" or "code" and are listed as numeric in the summary. Such
# data should be categorical, not numeric. There also should not be any kind of identifier (household id, plot id, etc)
# besides possibly "hhid".
print('variables with "id" or "code" and listed numeric:')

filtered_variables = summary[
    (summary["variable_name"].str.contains("id|code", case=False, na=False)) &
    (summary["data_type"] == "numeric")
]

# Print the name and description of the filtered variables
for _, row in filtered_variables.iterrows():
    print(f"Name: {row['variable_name']}, Description: {row['variable_description']}")

country: Guatemala
containing the word "unit":


,variable_name,data_type,module_name,module_description,variable_description,geographic_indicator
86,consumption_per_capita_per_day,numeric,CONSUMO5.DTA,Total consumption per household,Per-capita per-day household consumption adjus...,False
174,has_paved_or_gravel_road,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has a paved or gravel ro...,False
175,has_dirt_road,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has a dirt road accessible,False
176,has_footpath_no_gravel,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has a footpath without g...,False
177,has_footpath,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has a footpath (any kind...,False
178,has_sea_lake_river,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has access to transporta...,False
179,has_train,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has access to railway/tr...,False
180,has_airplane,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has access to air transp...,False
181,has_other,categorical,ECOM02.DTA,"road availability and common community uses, l...",Whether the community has access to any other ...,False


containing the word "consumption":


,variable_name,data_type,module_name,module_description,variable_description,geographic_indicator
86,consumption_per_capita_per_day,numeric,CONSUMO5.DTA,Total consumption per household,Per-capita per-day household consumption adjus...,False


variables with "id" or "code" and listed numeric:
Name: hhid, Description: household id
Name: num_video_camera, Description: Number of video cameras
Name: distance_traditional_midwife, Description: Distance to the  traditional midwife


## Summary correctness: Matches data, format

In [90]:
print(f'country: {country}')

# check that metadata and data match. It's OK if certain special column are not in summary: hhid, headcount_adjusted_hh_wgt,
# hh_wgt, consumption_per_capita_per_day. But it's also fine if they are.
data_columns = set(data.columns)

summary_variable_names = set(summary['variable_name'])
missing_in_data = summary_variable_names - data_columns
missing_in_summary = data_columns - summary_variable_names

print("Variables in summary but not in data:", missing_in_data)
print("Columns in data but not in summary:", missing_in_summary)

country: Guatemala
Variables in summary but not in data: set()
Columns in data but not in summary: set()


In [91]:
# Check that "summary" fits the required format, and datatypes are correct.


print(f'country: {country}')
required_columns = {
    "variable_name", "data_type", "geographic_indicator"
    }
summary_columns = set(summary.columns)

missing_columns = required_columns - summary_columns
if missing_columns:
    print(f"Missing required columns in summary: {missing_columns}")

# Ensure "data_type" has only permitted values
permitted_data_types = {"numeric", "categorical"}
for _, row in summary.iterrows():
    if row["data_type"] not in permitted_data_types:
        print(
            f"Invalid data_type '{row['data_type']}' for variable '{row['variable_name']}'. "
            f"Description: {row['variable_description']}"
        )

for _, row in summary.iterrows():
    for c in ["geographic_indicator"]:
        if c not in row:
            continue
        if row[c] not in [0, 1, True, False, None]:
            print(
                f"Invalid {c} '{row[c]}' for variable '{row['variable_name']}'. "
                f"Description: {row['variable_description']}"
            )

country: Guatemala


## Data types

In [92]:
# It's a big issue if a column is numeric in summary and non-numeric in data. It's not necessarily a problem if data
# is categorical in summary and numeric in data, because it might be a code/indicator; just take a quick scan that none of
# those columns seem like counts/numbers.
print(f'country: {country}')

# Check that numeric columns in summary are actually numeric in data
numeric_columns = summary[summary["data_type"] == "numeric"]["variable_name"]
for col in numeric_columns:
    if col in data.columns and not pd.api.types.is_numeric_dtype(data[col]):
        description = summary.loc[summary["variable_name"] == col, "variable_description"].values[0]
        print(f"BAD: numeric in summary, non-numeric in data: '{col}'; {description}")

# Check that categorical columns in summary are actually categorical in data (less important)
categorical_columns = summary[summary["data_type"] == "categorical"]["variable_name"]
for col in categorical_columns:
    if col in data.columns and not pd.api.types.is_categorical_dtype(data[col]):
        description = summary.loc[summary["variable_name"] == col, "variable_description"].values[0]
        print(f"categorical in summary, numeric in data: '{col}'; {description}")

country: Guatemala
categorical in summary, numeric in data: 'region'; administrative political region
categorical in summary, numeric in data: 'area'; urban or rural area
categorical in summary, numeric in data: 'p01a01'; type of housing
categorical in summary, numeric in data: 'p01a02'; wall material
categorical in summary, numeric in data: 'p01a03'; roof material
categorical in summary, numeric in data: 'p01a04'; floor material
categorical in summary, numeric in data: 'p01a05a'; water connection
categorical in summary, numeric in data: 'p01a05b'; drain connection
categorical in summary, numeric in data: 'p01a05c'; electricity connection
categorical in summary, numeric in data: 'p01a05d'; phone connection
categorical in summary, numeric in data: 'p01a05e'; water meter
categorical in summary, numeric in data: 'p01a05f'; electricity meter
categorical in summary, numeric in data: 'p01a10'; cooking place
categorical in summary, numeric in data: 'p01a11'; exclusive kitchen
categorical in s

C:\Users\PB\AppData\Local\Temp\ipykernel_6960\3292187331.py:16: DeprecationWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, pd.CategoricalDtype) instead
  if col in data.columns and not pd.api.types.is_categorical_dtype(data[col]):


## Duplicate columns

In [93]:
# This will print constant columns and equivalent columns. It's fine if there are some.
# 
# Constant columns sometimes indicate errors - if something seems like it should not be constant, but is, 
# that's probably an error. If two columns are printed
# as being equivalent and their name/description suggests they are duplicates, e.g. "state" and "state_code", we should ask them 
# to drop one. I expect many missingness-indicator columns to be equivalent to each other (for data that has little to no
# missingness). That's fine.

print(f'country: {country}')

find_equivalent_columns(data, summary)

country: Guatemala


Constant columns:
num_harvester



[]

In [94]:
data['num_harvester'].value_counts()

num_harvester
0.0    7276
Name: count, dtype: int64

## Geography and stratification

In [96]:
# Check that the rows identified as geographic indicators make sense: are all actually geo indicators, and nothing is missing.

print(f'country: {country}')

print('geographic indicators:')
display(summary[summary.geographic_indicator])
for _, row in summary[summary.geographic_indicator].iterrows():
    print(row.variable_name)
    print(data[row.variable_name].nunique())


country: Guatemala
geographic indicators:


,variable_name,data_type,module_name,module_description,variable_description,geographic_indicator
0,region,categorical,HOGARES.DTA,Module description,administrative political region,True
82,depto,categorical,CONSUMO5.DTA,Total consumption per household,department,True
83,mupio,numeric,CONSUMO5.DTA,Total consumption per household,municipality,True
84,sector,numeric,CONSUMO5.DTA,Total consumption per household,sector number,True


region
8
depto
22
mupio
35
sector
119
